In [9]:
%matplotlib inline
import tensorflow as tf
from vahun.Text import SplitBrainCorpus
import numpy as np
from vahun.tools import Timer
from vahun.tools import explog
from vahun.autoencoder import Autoencoder_ffnn
from vahun.tools import show_performance
from vahun.genetic import Settings
from vahun.tools import get_reconstruction
timer=Timer()



corpus_path='/mnt/store/velkey/mnsz2/brain_split.200k.maxlen20'
encode=200
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Xcorpus=SplitBrainCorpus(corpus_path=corpus_path,col=0,size=0)

Ycorpus=SplitBrainCorpus(corpus_path=corpus_path,col=1,size=0)

logger=explog(encoder_type="demo_autoencoder_splitbrain"+str(encode),
              encoding_dim=encode,
              feature_len=20,
              lang=corpus_path,
              unique_words=len(set(Xcorpus.wordlist)),
              name="demo_autoencoder_top_splitbrain"+str(encode),
              population_size=0,
              words=len(Xcorpus.wordlist))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


In [13]:
Ycorpus.wordlist[100]==Xcorpus.wordlist[100]

True

In [2]:
encoder=Autoencoder_ffnn(experiment=Settings([encode,len(Xcorpus.abc)*20]),
                 logger=logger,tf_session=sess,
                 inputdim=len(Xcorpus.abc)*20,
                 layerlist=[encode,len(Xcorpus.abc)*20],
                 encode_index=int(1),corpus=Xcorpus,
                 optimizer =tf.train.AdamOptimizer(learning_rate = 0.001),
                 nonlinear=tf.sigmoid,disp_step=800,
                 charnum=len(Xcorpus.abc))

In [3]:
encoder.train(Ycorpus.x_train,Ycorpus.x_valid,Ycorpus.x_test,512,30,X_train=Xcorpus.x_train)

In [4]:
result=get_reconstruction(encoder,Xcorpus.x_test,Xcorpus,20,len(Xcorpus.abc))

In [5]:
with open('/mnt/store/velkey/splitbrain_200_encoding', "a") as myfile:
    for it in result:
        string=""
        string+=str(it[0])
        string+='\t'
        string+=str(it[1])
        string+='\t'
        string+=str(it[2])
        string+='\n'
        myfile.write(string)

In [8]:
stds=show_performance(encoder,Xcorpus.x_test,Xcorpus,20,printer=True,inputdepth=20,inputfsize=len(Xcorpus.abc))

  _a_y_r_u_b_l_._u_n 	   _e_s_s_n_p_l_o_v_n 	 7
            k_n_í_n_ 	             k_n_m_n_ 	 1
          _m_e_z_e_t 	           _m_e_z_e_t 	 0
            _l_j_e_i 	             _l_j_e_i 	 0
      k_m_e_e_c_á_u_ 	       k_m_e_e_í_á_u_ 	 1
    r_c_o_a_i_á_á_á_ 	      _l_a_a_i_á_á_á_ 	 3
       _e_v_l_s_l_i_ 	        _e_v_l_s_l_i_ 	 0
     _o_m_n_p_r_o_r_ 	      _e_m_n_p_r_o_r_ 	 1
        b_r_d_l_m_a_ 	         k_r_d_l_m_a_ 	 1
            r_g_ó_r_ 	             r_g_ó_r_ 	 0
         _é_e_é_e_r_ 	          _é_e_é_e_r_ 	 0
         z_l_f_l_l_t 	          z_l_p_l_l_t 	 1
         m_g_l_ó_e_y 	          m_g_l_í_e_ő 	 2
        _o_s_t_s_n_k 	         _o_s_t_s_n_k 	 0
       _i_e_ö_é_é_é_ 	        _i_e_y_é_é_é_ 	 1
        l_g_i_á_y_t_ 	         l_g_i_á_y_t_ 	 0
               s_p_a 	                s_v_a 	 1
      o_o_t_l_f_n_ó_ 	        _g_t_l_v_n_ó_ 	 3
          k_c_k_s_j_ 	           k_p_k_s_j_ 	 1
            k_j_t_t_ 	             k_v_t_t_ 	 1

Accuracy on data:  95.73358225345926 %


In [12]:
std=show_performance(encoder,["eh","kecske","kutya","aytuk","macska","árvíztűrő","fúró","kacsa","a","és"],Xcorpus,printer=True,inputfsize=len(Xcorpus.abc))

ValueError: cannot reshape array of size 980 into shape (10,49)

In [ ]:
def decode_critical(lista,enc=180):
    for POS in lista:
        encoded=np.ones(enc)*-10
        encoded[POS]=10
        a=encoder.decode([encoded])[0].reshape([10,len(corpus.abc)])
        b=corp.defeaturize_data_charlevel_onehot([a])
        print(b)

In [ ]:
import heapq
topstd=heapq.nlargest(6, range(len(stds)), stds.__getitem__)
big_stuff=[]
for i in range(50):
    big_stuff.append([])
    for topind in topstd:
        a=encoder.encode([corpus.x_test[i]])
        a[0][topind]=2
        b=encoder.decode([a[0]])[0].reshape([10,len(corpus.abc)])
        c=corpus.defeaturize_data_charlevel_onehot([b])
        big_stuff[i].append(c)
for row in big_stuff:
    print(row)